In [19]:
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import nibabel as nib
from utils.preproc import FUSCleaner, load_data
from utils.glm import run_glm_analysis, compute_contrasts, plot_contrasts
import scipy.io as sio

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
load_dotenv()
base_dir = os.getenv('BASE_DIR')
zion_task_dir = os.path.join(base_dir, 'task_data', 'audiocue_acrossday', 'zion')
grouper_task_dir = os.path.join(base_dir, 'task_data', 'audiocue_acrossday', 'grouper')

In [31]:
data_dir = os.path.join(grouper_task_dir, '1002')
brain_file = os.path.join(data_dir, 'Grouper_1002_audiocue_plane5.5_2Dscan_1.nii')
event_file = os.path.join(data_dir, 'mldata.mat')

In [36]:
event_mat = sio.loadmat(event_file)['mldata'][0][0]

In [37]:
event_mat

(array([[1]], dtype=uint8), array([[1]], dtype=uint8), array([[1]], dtype=uint8), array([[1]], dtype=uint8), array([[2]], dtype=uint8), array([[9]], dtype=uint8), array([[nan]]), array([[0]], dtype=uint8), array([[2.0240e+03, 1.0000e+01, 2.0000e+00, 1.4000e+01, 3.1000e+01,
        1.6588e+01]]), array([[(array([[1.76337000e+01],
               [1.77768000e+02],
               [2.01813722e+04],
               [2.01963129e+04],
               [2.63270759e+04],
               [4.53453279e+04]]), array([[ 9],
               [ 0],
               [11],
               [ 2],
               [ 6],
               [18]], dtype=uint8))            ]],
      dtype=[('CodeTimes', 'O'), ('CodeNumbers', 'O')]), array([[(array([[1]], dtype=uint8), array([[ 1.15099304,  1.1641473 ],
               [ 1.38119165,  0.95368   ],
               [ 0.89448603,  1.01287393],
               ...,
               [-1.28253519, -0.190736  ],
               [-0.15785048, -0.26308415],
               [-0.01315421, -0.07

In [38]:
def extract_event_df(data):
    # Dictionary to store onsets for each trial type
    trial_onsets = {
        '5': [],
        '6': [],
        '7': []
    }

    for trial in data:
        absolute_time = trial["AbsoluteTrialStartTime"][0][0]
        code_times = trial["BehavioralCodes"][0]['CodeTimes'][0]
        code_numbers = trial["BehavioralCodes"][0]['CodeNumbers'][0]

        # Find the cue start time (CodeNumbers == 2)
        cue_idx = np.where(code_numbers == 2)[0]
        if len(cue_idx) > 0:
            cue_time = code_times[cue_idx[0]][0]
            cue_onset = absolute_time / 1000  # Convert to seconds
            print(f"Absolute trial start: {absolute_time/1000:.2f}s, Cue time: {cue_time/1000:.2f}s")
            
            # Find and store onsets for each trial type
            for trial_type in [5, 6, 7]:
                trial_idx = np.where(code_numbers == trial_type)[0]
                if len(trial_idx) > 0:
                    trial_time = code_times[trial_idx[0]][0]
                    # Use cue_onset instead of adding cue_time
                    trial_onsets[str(trial_type)].append(cue_onset)
                    print(f"  Trial type {trial_type}: time = {trial_time/1000:.2f}s, onset = {cue_onset:.2f}s")

    # Create DataFrame with trial types as rows and onsets as columns
    max_trials = max(len(onsets) for onsets in trial_onsets.values())
    df = pd.DataFrame(index=['5', '6', '7'])
    
    # Add columns for each trial
    for i in range(max_trials):
        df[f'trial_{i+1}'] = [
            trial_onsets['5'][i] if i < len(trial_onsets['5']) else np.nan,
            trial_onsets['6'][i] if i < len(trial_onsets['6']) else np.nan,
            trial_onsets['7'][i] if i < len(trial_onsets['7']) else np.nan
        ]
    
    return df

In [39]:
event_df = extract_event_df(event_mat)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [30]:
event_df

,trial_1,trial_2,trial_3,trial_4,trial_5,trial_6,trial_7,trial_8,trial_9,trial_10,...,trial_55,trial_56,trial_57,trial_58,trial_59,trial_60,trial_61,trial_62,trial_63,trial_64
5,96.058238,121.390998,216.542362,265.053447,334.922773,385.571365,438.090263,486.584656,507.942895,581.803335,...,3083.48982,3159.18716,3205.811277,3227.10272,3349.55775,3545.839843,3571.139109,3596.471733,3621.770993,3647.070251
6,70.692310,313.581251,360.222053,556.470714,651.656014,672.997508,698.346864,745.004330,838.252513,927.509636,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.000000,47.163516,142.883923,170.101986,193.330526,241.841607,290.352731,410.870603,459.398414,533.225439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
event_df.to_csv(os.path.join(data_dir, 'Zion_100324_audiocue_events.txt'), sep=' ', header=False, index=False)

In [9]:
event_df

,trial_1,trial_2,trial_3,trial_4,trial_5,trial_6,trial_7,trial_8,trial_9,trial_10,...,trial_54,trial_55,trial_56,trial_57,trial_58,trial_59,trial_60,trial_61,trial_62,trial_63
5,20.210047,47.680122,103.254973,131.276217,243.361107,325.938546,382.749137,526.796258,555.402220,579.081370,...,2967.447631,3024.258298,3158.085411,3214.545381,3265.093685,3288.489275,3494.77412,3517.885764,3545.105411,3627.816426
6,185.164536,271.482517,354.494135,746.373774,1150.861220,1198.938201,1252.809762,1687.288861,2251.771193,2333.864265,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,72.895916,155.206234,213.386089,299.971315,406.328374,436.470412,466.462117,496.737775,636.994133,690.631878,...,3420.446096,3445.611778,3572.492076,3598.492668,3650.811221,3675.993534,NaN,NaN,NaN,NaN


In [16]:
brain_img = nib.load(brain_file)
brain_data = brain_img.get_fdata()
brain_data.shape

pixdim[1,2,3] should be positive; setting to abs of pixdim values


(128, 1, 153, 9000)

In [17]:
9000 / 4

2250.0